<a href="https://colab.research.google.com/github/pragneshrana/Algo_Trial/blob/master/Fuel_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Calling libraries  and dataset 

In [18]:
#loaded data
import pandas as pd
url = 'https://raw.githubusercontent.com/sivaramambikasaran/Data_driven_Kinetics/master/data/Alkane_Dataset_full_stanford_AR.csv?token=AINCVGUITKDVFRUDJQI7RZS6HP64A'
df = pd.read_csv(url)# Dataset is now stored in a Pandas Dataframe
print(df.columns)
df = df.loc[:5,:]
print(df)

Index(['Fuel', 'T(K)', 'T_Error(%)', 'P(atm)', 'P_Error(%)', 'Fuel(%)',
       'Oxidizer(%)', 'Diluant(%)', 'Equv(phi)', 'Diluant Type', 'Time(μs)',
       'Mode_of_measurement', 'Species_name', 'Species_measurement_Error',
       'Measured_wavelength(nm)', 'Shocktube_dia(cm)', 'Research_group',
       'Data_Source'],
      dtype='object')
       Fuel  ...                                        Data_Source
0  CCCCCCCC  ...  Lam_2013_thesis / Normal_Alkanes_and_Simple_Ox...
1  CCCCCCCC  ...  Lam_2013_thesis / Normal_Alkanes_and_Simple_Ox...
2  CCCCCCCC  ...  Lam_2013_thesis / Normal_Alkanes_and_Simple_Ox...
3  CCCCCCCC  ...  Lam_2013_thesis / Normal_Alkanes_and_Simple_Ox...
4  CCCCCCCC  ...  Lam_2013_thesis / Normal_Alkanes_and_Simple_Ox...
5  CCCCCCCC  ...  Lam_2013_thesis / Normal_Alkanes_and_Simple_Ox...

[6 rows x 18 columns]


In [9]:
print(df['Fuel'] )

0      CCCCCCCC
1      CCCCCCCC
2      CCCCCCCC
3      CCCCCCCC
4      CCCCCCCC
         ...   
444     CCCCCCC
445     CCCCCCC
446     CCCCCCC
447     CCCCCCC
448     CCCCCCC
Name: Fuel, Length: 449, dtype: object


#Fucntion for finding the S_H bonds

In [28]:
def find_chain_length(fuel_data):
  '''
  This method will return the number of double bonds in the alkanes
  Input: list of fuel
  output : fuel S_H bond array 
  '''
  S_H = []
  #number bonds in the alkanes are (2n-2) for C>3 (S_H)
  for i in range(len(fuel_data)):
    S_H_count = len(fuel_data[i]) #count by fuel length
    S_H.append(S_H_count)
  return S_H

# #check fucntion
# print(find_chain_length(['CCC','CCCC']))


[3, 4]


#Required clumn from data

In [50]:
#calling library
import pandas as pd
import numpy as np

data = pd.DataFrame([]) #Empty DataFrame

#required data
data['S_H'] = find_chain_length(list(df['Fuel']))
data['T(K)'] = df['T(K)']
data['P(atm)'] = df['P(atm)']
data['Fuel(%)'] = df['Fuel(%)']
data['Oxidizer(%)'] = df['Oxidizer(%)']
data['Diluant(%)'] = df['Diluant(%)']
data['Equv(phi)'] = df['Equv(phi)']
data['Time(μs)'] = df['Time(μs)']
print(data)

   S_H  T(K)  P(atm)  Fuel(%)  Oxidizer(%)  Diluant(%)  Equv(phi)  Time(μs)
0    8  1325    1.95     0.32          4.0       95.68        1.0       859
1    8  1265    2.05     0.32          4.0       95.68        1.0      1569
2    8  1289    2.01     0.32          4.0       95.68        1.0      1198
3    8  1385    1.96     0.32          4.0       95.68        1.0       390
4    8  1455    1.87     0.32          4.0       95.68        1.0       166
5    8  1282    3.37     0.32          4.0       95.68        1.0      1020


# Conversion of data into the log or other form as per requirement to frame equation

In [51]:
converted_data  = pd.DataFrame([]) #Empty DataFrame

# conversion of parameters for the equation 
converted_data['S_H'] = find_chain_length(list(df['Fuel']))
converted_data['T'] = df['T(K)']
converted_data['log_T'] = np.log(df['T(K)'])
converted_data['log_P'] = np.log(df['P(atm)'])
converted_data['log_Fuel'] = np.log(df['Fuel(%)'])
converted_data['log_oxi'] = np.log(df['Oxidizer(%)'])
converted_data['log_Dilu'] = np.log(df['Diluant(%)'])
# converted_data['log_Equi'] = np.log(df['Equv(phi)'])
converted_data['log_Tim'] = np.log(df['Time(μs)'])
converted_data.columns

Index(['S_H', 'T', 'log_T', 'log_P', 'log_Fuel', 'log_oxi', 'log_Dilu',
       'log_Tim'],
      dtype='object')

#Mathematical operations

In [52]:
#Matrix and vector 
X = converted_data.iloc[:,:-1]
y =  converted_data.iloc[:,-1]
print(y)

0    6.755769
1    7.358194
2    7.088409
3    5.966147
4    5.111988
5    6.927558
Name: log_Tim, dtype: float64


In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train)

   S_H     T     log_T     log_P  log_Fuel   log_oxi  log_Dilu
5    8  1282  7.156177  1.214913 -1.139434  1.386294  4.561009
2    8  1289  7.161622  0.698135 -1.139434  1.386294  4.561009
4    8  1455  7.282761  0.625938 -1.139434  1.386294  4.561009
3    8  1385  7.233455  0.672944 -1.139434  1.386294  4.561009


#Regression
To solve $X\beta= y$

$X^TX \beta= X^Ty$

Now decomposing matrix using QR decomposition \
taking, $X = QR$

$(QR)^T(QR) \beta= (QR)^Ty$

$( R^T Q^T Q R) \beta= (QR)^Ty$

$( R^T R) \beta= R^T Q^Ty$

$ (R^T)^{-1} R^T R \beta=  Q^Ty$

$ R \beta=  Q^Ty$

$ R \beta=  z$

As R is upper triangular which can be solved by back substituion.


In [82]:

#Find parameters by regression using least-square formula (XT X)^-1 XT 
import scipy as sp

def back_substitution(A,b):
  '''
  This method will de backsubstitution
  '''
  n = b.size
  x = np.zeros(n)

  if(A[n-1, n-1] == 0):
      raise ValueError

  for i in range(n-1, 0, -1):
      x[i] = A[i, i]/b[i]
      for j in range (i-1, 0, -1):
          A[i, i] += A[j, i]*x[i]

  return x

def regression(X,y):
  '''
  This method will best fit using qr factorization
  '''
  Q,R = sp.linalg.qr(X)
  #solve 
  z = np.matmul(np.transpose(Q),y)
  #back-substitution
  beta = back_substitution(R,z)
  print('beta',beta)

regression(X,y)



beta [ 0.00000000e+00 -8.78199878e+01  2.45592262e-02 -1.98779465e+00
 -1.59277769e-14 -5.26006244e-30]


In [80]:
from sklearn.linear_model import LinearRegression as LR
reg = LR().fit(X, y)
reg.coef_

array([-4.40781682e-11, -3.31171382e-02,  2.85352574e+01, -5.03070074e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])